In [1]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)
import re

In [2]:
import numpy as np
import pandas as pd
import json
import os, sys

In [3]:
main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Prize Picks Projections Scrape

In [4]:
f = open(main_dir + "/PrizePicks/projections.json")
data = json.load(f)
pp = pd.json_normalize(data['included'])
pp1 = pp[pp['type'] == 'new_player']
pp2 = pd.json_normalize(data['data'])

pp1 = pp1.rename(columns = {'id' : 'playerid'})
pp2 = pp2.rename(columns = {'relationships.new_player.data.id' : 'playerid'})

mergedDF = pp2.merge(pp1, on = 'playerid', how = 'left')

fdf = mergedDF[['attributes.line_score','attributes.stat_type','attributes.name','attributes.league']]


In [5]:
fdf.rename(columns={'attributes.line_score':'pp_line', 'attributes.stat_type':'prop_type', 'attributes.name':'player_name', 
                   'attributes.league':'league'}, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
fdf['player_name'] = fdf['player_name'].str.lower()
fdf['league'] = fdf['league'].str.lower()
fdf['prop_type'] = fdf['prop_type'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [7]:
# changing prop types to match pinnacle
# keep adding to this until complete

# NFL PROPS
fdf['prop_type'] = np.where(fdf['prop_type']=='pass yards', 'pass yds', 
                           np.where(fdf['prop_type']=='receiving yards', 'receiving yds', 
                                    np.where(fdf['prop_type']=='rush yards', 'rush yds',
                                            fdf['prop_type'])))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Pinnacle Sports NFL WebScrape Attempt

In [8]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NFL/nfl_matchups.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()
#print(count)

In [9]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NFL/nfl_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()
#print(count)

In [10]:
propd = pd.DataFrame(player_props)

In [11]:
prices = pd.DataFrame(prop_prices)

In [12]:
df = pd.merge(propd, prices, how='left', on=['prop_id', 'over_id', 'under_id'])

In [13]:
df.dropna(inplace=True)

In [14]:
df['min_price'] = np.where(df['over_price']<=df['under_price'], df['over_price'], df['under_price'])

In [15]:
def prop_type(string):
    type_ = re.findall('(.+?)\(', string)[1].lower().strip(')')
    return type_

def player_name(string):
    name = re.findall('(.+?)\(', string)[0].lower().strip()
    return name

In [16]:
df['player_name'] = df['prop_name'].apply(player_name)
df['prop_type'] = df['prop_name'].apply(prop_type)

In [17]:
df.drop(columns='prop_name', inplace=True)

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
propd = pd.DataFrame(player_props)
prices = pd.DataFrame(prop_prices)

df = pd.merge(propd, prices, how='left', on=['prop_id', 'over_id', 'under_id'])
df.dropna(inplace=True)

df['min_price'] = np.where(df['over_price']<=df['under_price'], df['over_price'], df['under_price'])

df['player_name'] = df['prop_name'].apply(player_name)
df['prop_type'] = df['prop_name'].apply(prop_type)

df.drop(columns='prop_name', inplace=True)
df.drop_duplicates(inplace=True)

In [20]:
def get_picks(pp, pin, league):
    df = pp[pp['league']==league]
    df = df.merge(pin, how='left', on=['player_name', 'prop_type'])
    df.dropna(inplace=True)
    df.drop(columns=['prop_id', 'over_id', 'under_id', 'under_points'], inplace=True)
    df.rename(columns={'over_points':'pinnacle_line'}, inplace=True)
    
    df['favored'] = np.where(df['min_price']==df['over_price'], 'over', 'under')
    
    df['edge'] = np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='under'), 'edge',
                         np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='over'), 'edge',
                                 np.where(df['pinnacle_line']==df['pp_line'], 'equal',
                                         np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='over'), 'buffer',
                                                 np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='under'), 'buffer', 'else')))))
    df.sort_values(by='min_price', inplace=True)
    return df

In [21]:
nfl = get_picks(fdf, df, 'nfl')

In [22]:
nfl.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
16,2.0,receptions,tre'quan smith,nfl,2.5,128.0,-173.0,-173.0,under,buffer
13,2.0,receptions,marquez callaway,nfl,2.5,111.0,-148.0,-148.0,under,buffer
0,30.5,rush yds,kyler murray,nfl,29.5,-125.0,-106.0,-125.0,over,buffer
7,32.5,receiving yds,alvin kamara,nfl,33.5,-123.0,-108.0,-123.0,over,edge
4,65.5,rush yds,alvin kamara,nfl,60.5,-112.0,-119.0,-119.0,under,edge
9,47.5,receiving yds,zach ertz,nfl,47.5,-118.0,-112.0,-118.0,over,equal
14,48.5,receiving yds,rondale moore,nfl,48.5,-112.0,-118.0,-118.0,under,equal
15,27.5,receiving yds,tre'quan smith,nfl,27.5,-112.0,-118.0,-118.0,under,equal
18,5.5,receiving yds,keith kirkwood,nfl,5.5,-118.0,-112.0,-118.0,over,equal
8,60.5,receiving yds,chris olave,nfl,60.5,-113.0,-117.0,-117.0,under,equal
